In [8]:
# Решето Эратосфена
def generate_prime_table(n):
    multiples = set()
    for i in range(2, n+1):
        if i not in multiples:
            yield i
            multiples.update(range(i*i, n+1, i))

# subexponentional funtion
def L(c, g, q): return e^(c*(log(q))^g *(log(log(q)))^(1-g))

In [9]:
def check_factor(number, primeTable):
    
    def miller_rabin(n):
        with proof.WithProof('arithmetic',False):
            return is_prime(n)
    
    if (miller_rabin(number) and number not in primeTable):
        return False, None
    
    
    # we should not use only factor function
    # 1. we should try Miller-Rabin test [done]
    # 2. we should try Brenet's algoritm (or its analog like Lenstra alg) [TODO]
    #-----------------------------------
    decomposition = dict(factor(Integer(number)))
    # ----------------------------------
    numberPrimeFactors = set(decomposition.keys())
    success = (numberPrimeFactors.issubset(primeTable))
    
    return (success, decomposition)
        

def generate_equasion(a, q, primeTable, ring):
    
    equasionGenerated = False
    while not equasionGenerated:
        xsi = randint(2, q-1)
        axsi = ring(a)^xsi
        equasionGenerated, decomposition = check_factor(axsi, primeTable)
        
    coefficients = [decomposition[p] if p in decomposition else 0 for p in primeTable]
    
    return (xsi, coefficients)

In [10]:
def adleman_log(a, b, p): # return log_a (b) (mod p)
    if (p not in Primes()):
        return None
    
    ring = IntegerModRing(p)
    q = ring(a).multiplicative_order()
    ring_q = IntegerModRing(q)
    # TO DO: when q=p1^k1*...*ps^k2
    #        use Chinese remainder theorem
    if (q not in Primes()):
        return None
    
    # if both q and p are small we can find solution
    # with simple brute force
    if (q < sqrt(p) and p < 2^17):
        ax = ring(1)
        for x in range(1, q):
            ax *= a
            if (ax == b):
                return x
        return None
    
    # it is taken like this because of complexity balance on a lagre sacels
    border = ceil(L(1/2, 1/2, q)) 
    primeTable = set(generate_prime_table(border))
    
    coefficients = []
    equasions = []
    tableSize = len(primeTable)
    while len(equasions) != tableSize: 
        equasions += [generate_equasion(a,q,primeTable,ring) for _ in range(tableSize)]
        coefficients = list(map(lambda x: x[1], equasions))
        
        # USE Chinese remainder theorem SOMEHOWE when q is not prime
        independentEquasions = Matrix(ring_q, coefficients).transpose().pivots()
               
        equasions = [ equasions[i] for i in independentEquasions]
        coefficients = [ coefficients[i] for i in independentEquasions]
        
    # at this point equasions are generated, so we're solving them
    y = vector(list(map(lambda x: x[0], equasions)))
    
    A = Matrix(ring_q, coefficients)
    x = (A^-1) * y
    
    # finding log_a (b)
    while True:
        xsi = randint(2, q-1)
        bxsi = ring(b)^xsi
        found, decomposition = check_factor(bxsi, primeTable)
        if found:
            
            b_coeffisients = vector([decomposition[p] if p in decomposition else 0 for p in primeTable])
            
            return (ring_q(xsi)^-1)*(x.dot_product(b_coeffisients))

In [11]:
adleman_log(14306, 46388, 65521)

5

In [12]:
# 81119 - это простое число софи жермен порядка 2^16
adleman_log(2, 8192, 81119)

13

In [13]:
# 61209301350421962503 - это простое число софи жермен порядка 2^64
# adleman_log(2, 4611686018427387904, 61209301350421962503)